In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../db/words.csv')
dfw = df

In [4]:
df = pd.read_csv('../db/words.csv')

def split(word):
        return [char for char in word]
    
    
def remove_nonmatch(matchlist, dataframe): 
    #digs through the list of available words, if all the letters in the list are found, it keeps the word
    for i in matchlist:
        mask1 = dataframe.Words.str.contains(i)
        dataframe = dataframe.mask(~mask1).dropna()
    return dataframe

def remove_incorrect(matchlist, dataframe):   
    # checks the black list and removes any words containing any of those characters
    for i in matchlist:
        mask1 = dataframe.Words.str.contains(i)
        dataframe = dataframe.mask(mask1).dropna()
    return dataframe    


def wordle_solver(dataf):
    
    dfw = df

    while len(dfw) > 1:
        
        yellow = []
        yellow_index = []
        green = []
        green_index = []
        black = []
        black_index = []
        black_match = []
        black_match_index = []

        #take user input and asign it to be the core of this loop
        user_input = input('what word did you enter: ')
        CORES = split(user_input)
        set(CORES)

        # start of inputs
        yellow_input = input('which letters were yellow: ')

        # set to false for switching if no matches
        yellow_bool = False

        # split yellow letters to be indexed
        yellow_split = split(yellow_input)

        # append master list with inputs
        for i in yellow_split:
            yellow.append(i)
            
        #creates the key value for which letter went where
        yellow_index_current = [i for i, item in enumerate(CORES) if item in yellow]
        for i in yellow_index_current:
            yellow_index.append(i)
        
        # if index has length then flip switch from above to true, thus signaling that the yellow lists should be processed
        if len(yellow_index_current) > 0:
            yellow_bool = True


        # take in which letters were green    
        green_input = input('which letters were green: ')

        # split the green letters into a list and append the master
        green_split = split(green_input)
        for i in green_split:
            green.append(i)
        # switch set to off unless an input was given
        green_bool = False
        green_index_current = [i for i, item in enumerate(CORES) if item in green]
        
        for i in green_index_current:
            green_index.append(i)
        if len(green_index_current) > 0:
            green_bool = True

        if yellow_bool == True:
            for i in CORES:
                if i not in yellow:
                    black.append(i)
        #if green_bool == True:
            #for i in CORES:
                #if i not in green:
                    #black.append(i)
                    
        # checks the list of green letters and if duplicates found, removes them from the black list and adds them to the black match list which functions differently.            
        if green_bool == True: 
            for i, n in enumerate(CORES): 
                switch = True
                for x, s in enumerate(green):
                    if n == s and int(green_index[x]) == i:
                        switch = False
                    else:
                        pass

                if switch == True:
                    black.append(n)
                    black_index.append(i)

            for ind, let in enumerate(black):
                if let in green:
                    black.remove(let)
                    black_match_index.append(black_index.pop(ind))
                    black_match.append(let) 
        
        
        # remove words that dont fit the criteria from the working dataframe
        dfw = remove_nonmatch(yellow, dfw)
        dfw = remove_nonmatch(green, dfw)
        dfw = remove_incorrect(black, dfw)
        

        #split words into individual columns
        dfw_split = dfw['Words'].apply(lambda x: pd.Series(list(x)))
        
        
        #remove words that have letters in the incorrect places
        for i, n in enumerate(yellow):
            dfw_split = dfw_split[~dfw_split[yellow_index[i]].isin([n])]
        for i, n in enumerate(black_match):
            dfw_split = dfw_split[~dfw_split[black_match_index[i]].isin([n])]
        for i, n in enumerate(green):
            dfw_split = dfw_split[dfw_split[green_index[i]].isin([n])]
        
        # joins words back into a single column
        dfwser = dfw_split.agg(''.join, axis=1)
        #cast to a dataframe with column title 'Words'
        dfw = dfwser.to_frame(name='Words')
            

        #random word that could be an answer based on previous attempts
        ran_viable_word = dfw_split.sample()
        
        #processes the word to display as a single string
        word_display = []
        for i in ran_viable_word:
            word_display.append(ran_viable_word[i].to_string(index=False))
        word_display_joined = ''.join(map(str, word_display))

        
        print(f'TRY: {word_display_joined}')


if __name__ == '__main__':
    wordle_solver(df)


TRY: spare
TRY: shire
